In [1]:
import pandas as pd
import numpy as np
import datetime
import warnings
import joblib
warnings.filterwarnings('ignore')

In [151]:
# 头条source
source_lauch_info = pd.read_csv('./source_lauch_info.csv')
source_pay_sum_info = pd.read_csv('./source_pay_sum_info.csv')
plan_info = pd.read_csv('./plan_info.csv')
source_retain_info = pd.read_csv('./source_retain_info.csv')
source_pay_sum_1d_7d_info = pd.read_csv('./source_pay_sum_1d_7d_info.csv',index_col=0)

In [339]:
source_lauch_info.head()

,tdate,platform,game_id,channel_id,source_id,amount,click_num,down_num,pv,create_role_num
40,2020-01-02,1,1000840,6867,251792,955.05,267,71,25852,65
52,2020-01-02,1,1000840,7162,245825,67.44,87,0,1449,0
53,2020-01-02,1,1000840,7162,245826,606.73,794,29,36931,27
126,2020-01-02,1,1000840,20754,251672,1.53,2,1,630,2
127,2020-01-02,1,1000840,20754,251673,7.60,24,3,2438,3


In [340]:
source_pay_sum_info.head()

,tdate,channel_id,source_id,create_role_30d_money_sum
0,2020-06-21,-1,-1,117608
142,2020-06-21,6770,214140,0
143,2020-06-21,6770,214852,0
144,2020-06-21,6770,255940,0
145,2020-06-21,6770,257299,0


In [341]:
plan_info.head()

,plan_id,channel_id,source_id,image_id,launch_type,create_time
0,242954,20953,252731,18072.0,1,2020-01-02
1,242955,20953,252732,18563.0,1,2020-01-02
2,242956,20953,252733,19254.0,1,2020-01-02
3,243021,20953,252798,16276.0,1,2020-01-02
4,243022,20953,252799,18435.0,1,2020-01-02


In [342]:
source_pay_sum_1d_7d_info.head()

,tdate,channel_id,source_id,create_role_1d_money_sum,create_role_2d_money_sum,create_role_3d_money_sum,create_role_4d_money_sum,create_role_5d_money_sum,create_role_6d_money_sum,create_role_7d_money_sum
0,2020-06-21,-1,-1,6206,11946,15596,19328,23854,26072,31208
1,2020-06-21,6770,214140,0,0,0,0,0,0,0
2,2020-06-21,6770,214852,0,0,0,0,0,0,0
3,2020-06-21,6770,255940,0,0,0,0,0,0,0
4,2020-06-21,6770,257299,0,0,0,0,0,0,0


In [152]:
source_lauch_info = source_lauch_info[source_lauch_info['amount']>0]
source_lauch_info['tdate'] = pd.to_datetime(source_lauch_info['tdate']).dt.date
source_lauch_info.drop(['sum_money','pay_num','new_pay_money','new_user_num','new_role_num','paper_use_amount'],axis=1,inplace=True)

In [153]:
plan_info.rename(columns={'chl_user_id':'channel_id'},inplace=True)
# plan_info.rename(columns={'launch_time':'tdate'},inplace=True)
plan_info['create_time'] = pd.to_datetime(plan_info['create_time']).dt.date
plan_info = plan_info[['plan_id','channel_id','source_id','image_id','launch_type','create_time']]

In [154]:
plan_info.drop_duplicates(['channel_id','source_id'],inplace=True)

In [155]:
df = pd.merge(source_lauch_info,plan_info,on=['channel_id','source_id'],how='left',validate='many_to_one')

In [156]:
source_pay_sum_1d_7d_info.drop_duplicates(['channel_id','source_id','tdate'],inplace=True)
source_pay_sum_1d_7d_info['tdate'] = pd.to_datetime(source_pay_sum_1d_7d_info['tdate']).dt.date

In [157]:
df = pd.merge(df,source_pay_sum_1d_7d_info,on=['channel_id','source_id','tdate'],how='inner',validate='one_to_one')

In [158]:
source_pay_sum_info.rename(columns={'create_role_money_sum':'create_role_30d_money_sum'},inplace=True)
source_pay_sum_info['tdate'] = pd.to_datetime(source_pay_sum_info['tdate']).dt.date

In [159]:
source_pay_sum_info.drop_duplicates(['channel_id','source_id','tdate'],inplace=True)

In [160]:
df = pd.merge(df,source_pay_sum_info,on=['channel_id','source_id','tdate'],how='inner',validate='one_to_one')

In [161]:
df = df[df['tdate']>=datetime.date(2020, 6, 27)]

In [162]:
df['amount'].sum()

17239372.830000002

In [163]:
df['create_role_30d_money_sum'].sum()

6143072

In [164]:
df['create_role_30d_money_sum'].sum()/df['amount'].sum()

0.35633964533267765

In [165]:
df.head()

,tdate,platform,game_id,channel_id,source_id,amount,click_num,down_num,pv,create_role_num,plan_id,image_id,launch_type,create_time,create_role_1d_money_sum,create_role_2d_money_sum,create_role_3d_money_sum,create_role_4d_money_sum,create_role_5d_money_sum,create_role_6d_money_sum,create_role_7d_money_sum,create_role_30d_money_sum
1619,2020-06-27,1,1000840,6867,319099,38.87,91,1,5995,0,309321.0,22193.0,1.0,2020-06-13,0,0,0,0,0,0,0,0
1620,2020-06-27,1,1000840,6867,320861,2997.82,2780,203,226157,203,311083.0,22193.0,1.0,2020-06-20,0,0,0,0,0,0,0,0
1621,2020-06-27,1,1000840,6867,320862,175.83,184,6,14905,6,311084.0,22193.0,1.0,2020-06-20,0,0,0,0,0,0,0,0
1622,2020-06-27,1,1000840,6867,320863,1520.21,755,54,85105,51,311086.0,22281.0,1.0,2020-06-20,30,30,30,30,30,30,30,30
1623,2020-06-27,1,1000840,6867,320864,2508.70,1302,160,165748,157,311087.0,22326.0,1.0,2020-06-20,0,128,238,238,238,238,238,268


In [166]:
df.shape

(9924, 22)

In [167]:
source = pd.read_pickle('./source.pkl')
source = source[['channel_id','source_id','first_amount_time']]
source.drop_duplicates(inplace=True)
df = pd.merge(df,source,on=['channel_id','source_id'],how='left',validate='many_to_one')

In [168]:
source_run_data = pd.read_pickle('./source_run_data.pkl')

In [169]:
source_run_data = source_run_data[source_run_data['media_id']==10]

In [170]:
source_run_data.head()

,channel_id,source_id,create_time,first_amount_time,source_amount_sum,model_run_date,media_id,source_predict_pay,30_ROI
8,21116,316350,2020-06-02,2020-06-02,96.88,2020-07-01,10.0,0.000000,0.000000
9,21116,317661,2020-06-08,2020-06-08,3678.67,2020-07-01,10.0,724.536771,0.196956
10,20754,319040,2020-06-13,2020-06-13,4464.49,2020-07-01,10.0,1077.222834,0.241287
13,21116,319211,2020-06-13,2020-06-13,3.70,2020-07-01,10.0,0.000000,0.000000
14,20754,319825,2020-06-16,2020-06-16,2025.64,2020-07-01,10.0,2944.033856,1.453385


In [176]:
source_info_1 = df

In [191]:
def source_win(model_run_date):
    # 在2010-7-1号进行计划判断是，为了保证有1.2.3三种role_data_type的数据，应该取27-28-29这三天注册的用户，否定如果取28-29-30这三天注册的
    #用户，则只能有0.1.2三种role_data_type的数据
    source_info_1_run_data = source_info_1[(source_info_1['tdate']>=model_run_date - datetime.timedelta(4))
                                              &(source_info_1['tdate']<model_run_date - datetime.timedelta(1))]
    
    
    # 聚合计算窗口期每条计划的合计占击
    groups = source_info_1_run_data.groupby(['channel_id','source_id'])
    temp = groups['click_num'].agg([('click_num_sum','sum')]).reset_index()
    source_info_1_run_data = pd.merge(source_info_1_run_data,temp,on=['channel_id','source_id'],how='left',validate='many_to_one')
    
    # 聚合计算窗口期每条计划的合计down_num
    temp = groups['down_num'].agg([('down_num_sum','sum')]).reset_index()
    source_info_1_run_data = pd.merge(source_info_1_run_data,temp,on=['channel_id','source_id'],how='left',validate='many_to_one')
    
    # 聚合计算窗口期每条计划的合计pv
    temp = groups['pv'].agg([('pv_sum','sum')]).reset_index()
    source_info_1_run_data = pd.merge(source_info_1_run_data,temp,on=['channel_id','source_id'],how='left',validate='many_to_one')
    
    # 聚合计算窗口期每条计划的合计create_role_num
    temp = groups['create_role_num'].agg([('create_role_num_sum','sum')]).reset_index()
    source_info_1_run_data = pd.merge(source_info_1_run_data,temp,on=['channel_id','source_id'],how='left',validate='many_to_one')
    
    # 聚合计算窗口期每条计划的合计create_role_1d_money_sum
    temp = groups['create_role_1d_money_sum'].agg([('create_role_1d_money_sum_sum','sum')]).reset_index()
    source_info_1_run_data = pd.merge(source_info_1_run_data,temp,on=['channel_id','source_id'],how='left',validate='many_to_one')
    
    temp = groups['create_role_2d_money_sum'].agg([('create_role_2d_money_sum_sum','sum')]).reset_index()
    source_info_1_run_data = pd.merge(source_info_1_run_data,temp,on=['channel_id','source_id'],how='left',validate='many_to_one')
    
    temp = groups['create_role_3d_money_sum'].agg([('create_role_3d_money_sum_sum','sum')]).reset_index()
    source_info_1_run_data = pd.merge(source_info_1_run_data,temp,on=['channel_id','source_id'],how='left',validate='many_to_one')
    
    temp = groups['create_role_4d_money_sum'].agg([('create_role_4d_money_sum_sum','sum')]).reset_index()
    source_info_1_run_data = pd.merge(source_info_1_run_data,temp,on=['channel_id','source_id'],how='left',validate='many_to_one')
    
    temp = groups['create_role_5d_money_sum'].agg([('create_role_5d_money_sum_sum','sum')]).reset_index()
    source_info_1_run_data = pd.merge(source_info_1_run_data,temp,on=['channel_id','source_id'],how='left',validate='many_to_one')
    
    temp = groups['create_role_6d_money_sum'].agg([('create_role_6d_money_sum_sum','sum')]).reset_index()
    source_info_1_run_data = pd.merge(source_info_1_run_data,temp,on=['channel_id','source_id'],how='left',validate='many_to_one')
    
    temp = groups['create_role_7d_money_sum'].agg([('create_role_7d_money_sum_sum','sum')]).reset_index()
    source_info_1_run_data = pd.merge(source_info_1_run_data,temp,on=['channel_id','source_id'],how='left',validate='many_to_one')
    
    temp = groups['create_role_30d_money_sum'].agg([('create_role_30d_money_sum_sum','sum')]).reset_index()
    source_info_1_run_data = pd.merge(source_info_1_run_data,temp,on=['channel_id','source_id'],how='left',validate='many_to_one')
    
    source_info_1_run_data = source_info_1_run_data[['channel_id','source_id','platform','plan_id','image_id','launch_type','game_id',
                                                     'click_num_sum','down_num_sum','pv_sum','create_role_num_sum','create_role_1d_money_sum_sum',
                                                    'create_role_2d_money_sum_sum','create_role_3d_money_sum_sum','create_role_4d_money_sum_sum',
                                                    'create_role_5d_money_sum_sum','create_role_6d_money_sum_sum','create_role_7d_money_sum_sum',
                                                    'create_role_30d_money_sum_sum']].drop_duplicates()
    source_info_1_run_data['model_run_date'] = model_run_date
    

 
    return source_info_1_run_data

In [192]:
source_info_1_run_data = pd.DataFrame()
for i in range(1,31):   
    model_run_date = datetime.date(2020, 7, i)
    source_run_data_i = source_win(model_run_date=model_run_date)
    source_info_1_run_data = source_info_1_run_data.append(source_run_data_i)

In [193]:
source_info_1_run_data.head()

,channel_id,source_id,platform,plan_id,image_id,launch_type,game_id,click_num_sum,down_num_sum,pv_sum,create_role_num_sum,create_role_1d_money_sum_sum,create_role_2d_money_sum_sum,create_role_3d_money_sum_sum,create_role_4d_money_sum_sum,create_role_5d_money_sum_sum,create_role_6d_money_sum_sum,create_role_7d_money_sum_sum,create_role_30d_money_sum_sum,model_run_date
0,6867,319099,1,309321.0,22193.0,1.0,1000840,91,1,5995,0,0,0,0,0,0,0,0,0,2020-07-01
1,6867,320861,1,311083.0,22193.0,1.0,1000840,2780,203,226157,203,0,0,0,0,0,0,0,0,2020-07-01
2,6867,320862,1,311084.0,22193.0,1.0,1000840,184,6,14905,6,0,0,0,0,0,0,0,0,2020-07-01
3,6867,320863,1,311086.0,22281.0,1.0,1000840,755,54,85105,51,30,30,30,30,30,30,30,30,2020-07-01
4,6867,320864,1,311087.0,22326.0,1.0,1000840,1302,160,165748,157,0,128,238,238,238,238,238,268,2020-07-01


In [194]:
source_run_data.head()

,channel_id,source_id,create_time,first_amount_time,source_amount_sum,model_run_date,media_id,source_predict_pay,30_ROI
8,21116,316350,2020-06-02,2020-06-02,96.88,2020-07-01,10.0,0.000000,0.000000
9,21116,317661,2020-06-08,2020-06-08,3678.67,2020-07-01,10.0,724.536771,0.196956
10,20754,319040,2020-06-13,2020-06-13,4464.49,2020-07-01,10.0,1077.222834,0.241287
13,21116,319211,2020-06-13,2020-06-13,3.70,2020-07-01,10.0,0.000000,0.000000
14,20754,319825,2020-06-16,2020-06-16,2025.64,2020-07-01,10.0,2944.033856,1.453385


In [197]:
source_run_data = pd.merge(source_run_data,source_info_1_run_data,on=['channel_id','source_id','model_run_date'],how='left',validate='one_to_one')

In [199]:
source_run_data.shape

(15482, 26)

In [214]:
source_run_data.head()

,channel_id,source_id,create_time,first_amount_time,source_amount_sum,model_run_date,media_id,source_predict_pay,30_ROI,platform,plan_id,image_id,launch_type,game_id,click_num_sum,down_num_sum,pv_sum,create_role_num_sum,create_role_1d_money_sum_sum,create_role_2d_money_sum_sum,create_role_3d_money_sum_sum,create_role_4d_money_sum_sum,create_role_5d_money_sum_sum,create_role_6d_money_sum_sum,create_role_7d_money_sum_sum,create_role_30d_money_sum_sum,turn_off
0,21116,316350,2020-06-02,2020-06-02,96.88,2020-07-01,10.0,0.000000,0.000000,1.0,306572.0,21935.0,1.0,1000840.0,3.0,1.0,20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
1,21116,317661,2020-06-08,2020-06-08,3678.67,2020-07-01,10.0,724.536771,0.087532,1.0,307883.0,20819.0,1.0,1000840.0,1372.0,67.0,255145.0,65.0,12.0,114.0,182.0,182.0,194.0,194.0,194.0,322.0,1
2,20754,319040,2020-06-13,2020-06-13,4464.49,2020-07-01,10.0,1077.222834,0.804571,1.0,309262.0,22003.0,1.0,1000840.0,10498.0,318.0,1517446.0,300.0,42.0,42.0,304.0,316.0,514.0,612.0,770.0,3592.0,0
3,21116,319211,2020-06-13,2020-06-13,3.70,2020-07-01,10.0,0.000000,0.000000,1.0,309433.0,21388.0,1.0,1000840.0,39.0,19.0,8304.0,18.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
4,20754,319825,2020-06-16,2020-06-16,2025.64,2020-07-01,10.0,2944.033856,1.221342,1.0,310046.0,22326.0,1.0,1000840.0,707.0,110.0,60854.0,106.0,12.0,250.0,678.0,1022.0,1022.0,1090.0,1090.0,2474.0,0


In [201]:
source_run_data.dropna(how='any',inplace=True)

In [204]:
source_run_data['game_id'].nunique()

5

In [211]:
source_run_data['30_ROI'] = source_run_data['create_role_30d_money_sum_sum'] / source_run_data['source_amount_sum']

In [277]:
((source_run_data['model_run_date'] - source_run_data['first_amount_time']) > datetime.timedelta(3))

0         True
1         True
2         True
3         True
4         True
         ...  
15477    False
15478     True
15479    False
15480    False
15481    False
Length: 15358, dtype: bool

In [315]:
source_run_data['turn_off_roi'] = (source_run_data['30_ROI']<0.1).astype(np.int)
source_run_data['turn_off_time'] = ((source_run_data['model_run_date'] - source_run_data['first_amount_time']) > datetime.timedelta(3)).astype(np.int)

In [316]:
source_run_data['turn_off'] = source_run_data['turn_off_roi'] + source_run_data['turn_off_time']

In [317]:
# source_run_data['turn_off'] = source_run_data['turn_off_roi']

In [318]:
result = source_run_data[['channel_id','source_id','model_run_date','turn_off']]

In [319]:
result = result[result['turn_off']==2].sort_values('model_run_date')

In [320]:
result.drop_duplicates(['channel_id','source_id'],keep='first',inplace=True)

In [321]:
result.head(10)

,channel_id,source_id,model_run_date,turn_off
0,21116,316350,2020-07-01,2
330,20143,321210,2020-07-01,2
331,20755,321242,2020-07-01,2
333,20144,321443,2020-07-01,2
335,20755,321655,2020-07-01,2
336,20755,321656,2020-07-01,2
337,20541,321661,2020-07-01,2
338,20541,321662,2020-07-01,2
339,21117,321664,2020-07-01,2
340,21117,321666,2020-07-01,2


In [322]:
result.shape

(3182, 4)

In [323]:
result['model_run_date'].value_counts()

2020-07-20    193
2020-07-01    187
2020-07-15    174
2020-07-24    154
2020-07-23    148
2020-07-12    147
2020-07-13    142
2020-07-17    139
2020-07-02    137
2020-07-22    121
2020-07-21    112
2020-07-18    111
2020-07-25    108
2020-07-27    107
2020-07-26    102
2020-07-10    102
2020-07-11    101
2020-07-03    101
2020-07-19     98
2020-07-08     97
2020-07-06     96
2020-07-07     95
2020-07-05     91
2020-07-14     87
2020-07-04     67
2020-07-29     60
2020-07-16     43
2020-07-28     25
2020-07-30     21
2020-07-09     16
Name: model_run_date, dtype: int64

In [324]:
data = pd.read_csv('./data.csv',index_col=0)
data['channel_id'] = data['channel_id'].astype(np.int64)
data['source_id'] = data['source_id'].astype(np.int64)
for i in range(1,31):
    exec("data['time'] = data['time'].replace(i,'2020-07-%s')"% (i))
data['time'] = pd.to_datetime(data['time'])
data.rename(columns={'time':'source_run_date'},inplace=True)
data=data[data['amount']!=0]
data_merge = pd.merge(data,result,on=['channel_id','source_id'],how='left',validate='many_to_one')
data_merge.dropna(how='any',inplace=True)


In [325]:
data_merge.head()

,channel_id,source_id,30ROI,创角30日LTV,创角账号数,amount,source_run_date,pay_sum,model_run_date,turn_off
0,20540,323114,0.0,0.0,0.0,589.49,2020-07-01,0.0,2020-07-05,2.0
2,20953,323225,0.0,0.0,2.0,1071.33,2020-07-01,0.0,2020-07-05,2.0
3,21116,322970,0.0,0.0,1.0,523.95,2020-07-01,0.0,2020-07-04,2.0
4,20953,323072,0.0,0.0,1.0,500.96,2020-07-01,0.0,2020-07-05,2.0
5,20540,323112,0.0,0.0,0.0,494.91,2020-07-01,0.0,2020-07-05,2.0


In [326]:
data_merge['istrue'] = (data_merge['source_run_date']>=data_merge['model_run_date'])

In [327]:
data_merge_true=data_merge[data_merge['istrue']==1]
model_roi = round((data_merge_true['pay_sum'].sum())/(data_merge_true['amount'].sum()),4)

In [328]:
print(model_roi)

0.4827


In [329]:
data_merge_true['amount'].sum()

2044607.1799999997

In [330]:
(data['pay_sum'].sum()-data_merge_true['pay_sum'].sum())/(data['amount'].sum()-data_merge_true['amount'].sum())

0.32201604151223756

In [265]:
df = source_run_data[(source_run_data['channel_id'] == 20953.0) & (source_run_data['source_id'] == 332064)]

In [266]:
df

,channel_id,source_id,create_time,first_amount_time,source_amount_sum,model_run_date,media_id,source_predict_pay,30_ROI,platform,plan_id,image_id,launch_type,game_id,click_num_sum,down_num_sum,pv_sum,create_role_num_sum,create_role_1d_money_sum_sum,create_role_2d_money_sum_sum,create_role_3d_money_sum_sum,create_role_4d_money_sum_sum,create_role_5d_money_sum_sum,create_role_6d_money_sum_sum,create_role_7d_money_sum_sum,create_role_30d_money_sum_sum,turn_off
9441,20953,332064,2020-07-17,2020-07-17,5398.93,2020-07-19,10.0,2798.598143,0.350440,1.0,322284.0,23528.0,1.0,1000840.0,459.0,85.0,214442.0,84.0,190.0,416.0,526.0,956.0,1114.0,1380.0,1448.0,1892.0,0
10066,20953,332064,2020-07-17,2020-07-17,110265.11,2020-07-20,10.0,88936.308011,0.847249,1.0,322284.0,23528.0,1.0,1000840.0,12232.0,1815.0,2796585.0,1776.0,8262.0,19834.0,24904.0,28208.0,29704.0,30728.0,38662.0,93422.0,0
10681,20953,332064,2020-07-17,2020-07-17,118407.97,2020-07-21,10.0,115925.847189,0.789085,1.0,322284.0,23528.0,1.0,1000840.0,13089.0,1995.0,2971867.0,1954.0,8262.0,19834.0,24904.0,28220.0,29716.0,30740.0,38674.0,93434.0,0
11285,20953,332064,2020-07-17,2020-07-17,119560.88,2020-07-22,10.0,97889.625691,0.768178,1.0,322284.0,23528.0,1.0,1000840.0,13076.0,1982.0,2904266.0,1941.0,8254.0,19690.0,24650.0,27536.0,28874.0,29632.0,37498.0,91844.0,0
11916,20953,332064,2020-07-17,2020-07-17,18528.35,2020-07-23,10.0,917.857622,0.021481,1.0,322284.0,23528.0,1.0,1000840.0,2003.0,304.0,430683.0,301.0,212.0,302.0,356.0,368.0,368.0,368.0,368.0,398.0,1
12559,20953,332064,2020-07-17,2020-07-17,12282.96,2020-07-24,10.0,1241.871283,0.301230,1.0,322284.0,23528.0,1.0,1000840.0,1529.0,145.0,316440.0,147.0,382.0,472.0,594.0,594.0,594.0,594.0,594.0,3700.0,0
13160,20953,332064,2020-07-17,2020-07-17,8520.93,2020-07-25,10.0,683.852049,0.398783,1.0,322284.0,23528.0,1.0,1000840.0,1563.0,89.0,276344.0,92.0,200.0,200.0,322.0,322.0,322.0,322.0,322.0,3398.0,0
13710,20953,332064,2020-07-17,2020-07-17,7955.18,2020-07-26,10.0,616.682464,0.416584,1.0,322284.0,23528.0,1.0,1000840.0,1342.0,57.0,283010.0,61.0,170.0,170.0,238.0,238.0,238.0,238.0,238.0,3314.0,0
14158,20953,332064,2020-07-17,2020-07-17,11044.25,2020-07-27,10.0,0.000000,0.002716,1.0,322284.0,23528.0,1.0,1000840.0,1672.0,92.0,421675.0,86.0,0.0,0.0,0.0,0.0,0.0,30.0,30.0,30.0,1
14522,20953,332064,2020-07-17,2020-07-17,8254.44,2020-07-28,10.0,43.919487,0.003634,1.0,322284.0,23528.0,1.0,1000840.0,1192.0,76.0,314930.0,70.0,0.0,0.0,0.0,0.0,0.0,30.0,30.0,30.0,1


In [267]:
data[(data['channel_id'] == 20953.0) & (data['source_id'] == 332064)]

,channel_id,source_id,30ROI,创角30日LTV,创角账号数,amount,source_run_date,pay_sum
222,20953,332064,0.350440,22.523810,84.0,5398.93,2020-07-17,1892.000040
238,20953,332064,0.872827,54.095745,1692.0,104866.18,2020-07-18,91530.000540
303,20953,332064,0.001474,0.067416,178.0,8142.86,2020-07-19,12.000048
194,20953,332064,0.046094,4.253521,71.0,6551.84,2020-07-20,301.999991
247,20953,332064,0.021911,1.615385,52.0,3833.65,2020-07-21,84.000020
153,20953,332064,1.746536,138.083333,24.0,1897.47,2020-07-22,3313.999992
57,20953,332064,0.000000,0.000000,16.0,2789.81,2020-07-23,0.000000
35,20953,332064,0.000000,0.000000,21.0,3267.90,2020-07-24,0.000000
116,20953,332064,0.006016,0.612245,49.0,4986.54,2020-07-25,30.000005
90,20953,332064,0.000000,0.000000,15.0,1526.21,2020-07-27,0.000000


In [268]:
data_merge_true[(data_merge_true['channel_id'] == 20953.0) & (data_merge_true['source_id'] == 332064)]

,channel_id,source_id,30ROI,创角30日LTV,创角账号数,amount,source_run_date,pay_sum,model_run_date,turn_off,istrue
8499,20953,332064,0.000000,0.000000,16.0,2789.81,2020-07-23,0.000000,2020-07-23,1.0,True
8759,20953,332064,0.000000,0.000000,21.0,3267.90,2020-07-24,0.000000,2020-07-23,1.0,True
9043,20953,332064,0.006016,0.612245,49.0,4986.54,2020-07-25,30.000005,2020-07-23,1.0,True
9458,20953,332064,0.000000,0.000000,15.0,1526.21,2020-07-27,0.000000,2020-07-23,1.0,True


In [293]:
data_merge_true.sort_values('pay_sum',ascending=False).head(20)

,channel_id,source_id,30ROI,创角30日LTV,创角账号数,amount,source_run_date,pay_sum,model_run_date,turn_off,istrue
226,20755,322389,18.146644,221.840173,463.0,5660.11,2020-07-01,102712.000099,2020-07-01,2.0,True
3151,20754,322678,4.639232,37.678032,2317.0,18817.77,2020-07-10,87300.000144,2020-07-07,2.0,True
7817,20144,326813,55.439822,4604.000000,14.0,1162.63,2020-07-21,64456.000000,2020-07-18,2.0,True
2624,20755,324504,12.454156,454.644628,121.0,4417.16,2020-07-09,55011.999988,2020-07-07,2.0,True
8612,20953,330590,8.157530,478.018018,111.0,6504.42,2020-07-23,53059.999998,2020-07-20,2.0,True
6483,20755,322389,3.322939,152.318021,283.0,12972.25,2020-07-18,43105.999943,2020-07-01,2.0,True
2322,20754,322678,8.122146,75.269155,509.0,4716.98,2020-07-08,38311.999895,2020-07-07,2.0,True
9355,20754,330337,2.723768,83.518600,457.0,14012.94,2020-07-26,38168.000200,2020-07-19,2.0,True
1019,21117,322743,15.508201,651.843137,51.0,2143.64,2020-07-04,33243.999987,2020-07-03,2.0,True
8836,6770,332913,17.690932,1208.814815,27.0,1844.90,2020-07-24,32638.000005,2020-07-24,2.0,True


In [298]:
data[(data['channel_id'] == 20144) & (data['source_id'] == 326813)]

,channel_id,source_id,30ROI,创角30日LTV,创角账号数,amount,source_run_date,pay_sum
165,20144,326813,0.033181,4.000000,3.0,361.65,2020-07-09,12.000000
58,20144,326813,0.145906,30.588235,17.0,3563.94,2020-07-10,519.999995
41,20144,326813,0.785645,150.387097,31.0,5933.98,2020-07-12,4662.000007
101,20144,326813,1.865348,187.966102,59.0,5945.27,2020-07-13,11090.000018
85,20144,326813,0.002583,0.272727,44.0,4645.45,2020-07-14,11.999988
112,20144,326813,0.045478,4.380952,63.0,6068.82,2020-07-15,275.999976
201,20144,326813,0.000000,0.000000,45.0,4030.53,2020-07-16,0.000000
84,20144,326813,0.000000,0.000000,20.0,3233.21,2020-07-17,0.000000
94,20144,326813,0.003925,0.571429,21.0,3057.20,2020-07-18,12.000009
84,20144,326813,0.027521,3.813333,75.0,10391.90,2020-07-19,285.999975


In [332]:
df1 = data_merge_true[(data_merge_true['channel_id'] == 20754) & (data_merge_true['source_id'] == 322678)]

In [333]:
df1['amount'].sum()

64847.62

In [334]:
df1['pay_sum'].sum()

130977.99951700002

In [335]:
df2 = data_merge_true[(data_merge_true['channel_id'] == 20144) & (data_merge_true['source_id'] == 326813)]

In [336]:
df2['amount'].sum()

20986.01

In [337]:
df2['pay_sum'].sum()

65678.000006

In [338]:
(data['pay_sum'].sum()-data_merge_true['pay_sum'].sum()+196655)/(data['amount'].sum()-data_merge_true['amount'].sum()+85825)

0.3347157275529696